In [ ]:
 from google.colab import drive
 drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd

tf = pd.read_csv("gdrive/My Drive/data/shorelines/time-series-selection.csv")

print(f"Unique transects in TimeFrame {len(tf['transect_id'].unique())}")
print(f"Unique transects in metadata: {len(metadata)}")

Unique transects in TimeFrame 635648
Unique transects in metadata: 635648


In [ ]:
from datetime import timedelta, datetime
from tqdm.auto import tqdm

def partial2date(number, reference_year=1984):
  year = reference_year + int(number)
  d = timedelta(days=(reference_year + number - year)*365)
  day_one = datetime(year, 1, 1)
  date = d + day_one
  return date

def partials2dates(list_of_part_dates):
  return [partial2date(idx) for idx in list_of_part_dates]

tqdm.pandas()

tf['dt'] = partials2dates(tf['dt'])
tf = tf.set_index('dt')
tf = tf.groupby(['transect_id', tf.index.year]).mean().reset_index()
tf = tf.pivot(index='transect_id', columns='dt', values='dist')
# tf.to_csv("gdrive/My Drive/data/shorelines/time-series-gluonts-prepared.csv", index=True, header=True)
tf.head()

In [ ]:
tf = pd.read_csv("gdrive/My Drive/data/shorelines/time-series-gluonts-prepared.csv")
tf.head()


### Filter metadata

In [ ]:
# keep metadata which is in TimeFrame
print(f"Transects in TimeFrame: {len(tf['transect_id'].unique())}")
metadata = pd.read_pickle("gdrive/My Drive/data/shorelines/sites-compressed.pkl")
metadata = metadata.loc[metadata['transect_id'].isin(list(tf['transect_id'].unique()))]
print(f"Transects in Metadata: {len(metadata['transect_id'].unique())}")

In [7]:
# drop redundant columns (i.e., time series data)
metadata = metadata.drop(columns=['outliers_1', 'outliers_2', 'dt', 'dist'])

In [8]:
# drop rows without lon/lat 
import geopandas as gpd
from shapely.geometry import Point

def df2gdf(df):
  crs = {"init": "epsg:4326"}
  return gpd.GeoDataFrame(df, crs=crs, geometry=df['geometry'])

print(f"Transects in metadata: {len(metadata))}")
gdf = df2gdf(metadata)
keep_indices = gdf.loc[gdf['geometry'].is_empty == False]['transect_id'].to_list()
metadata = metadata.loc[metadata["transect_id"].isin(keep_indices)]
print(f"Transects after dropping empty lon/lat: {len(metadata))}")

# Save as pickle to keep optimized dataframe (less memory)
# metadata.to_picke("/media/storage/data/shorelines/sites-gluonts-prepared.pkl")

635551


### Update TimeFrame

In [10]:
tf.loc[tf['transect_id'].isin(list(metadata['transect_id'].unique()))]

# Save to csv
# tf.to_csv("/media/storage/data/shorelines/time-series-gluonts-prepared.csv", index=False, header=True)